In [ ]:
# xbotinterface imports (for modelinterface)
from xbot_interface import xbot_interface as xbot
from xbot_interface import config_options as co

# cartesio imports (for Affine3 and IK solver)
from cartesian_interface.pyci_all import *
import cartesian_interface.roscpp_utils as roscpp

# cartesio_planning imports
from cartesio_planning import planning
from cartesio_planning import validity_check
from cartesio_planning import visual_tools
from cartesio_planning import constraints

# linear algebra, yaml, ros, ...
import numpy as np
import scipy.linalg as la
import yaml
import moveit_msgs.msg 
import rospy
import StringIO


# we need to initialize roscpp (underlying code is in C++!)
cpp_argv = []
if not roscpp.init('teleop_example', cpp_argv):
    print 'Unable to initialize roscpp node!'

In [ ]:
# get robot description from param server
urdf = rospy.get_param('robot_description')
srdf = rospy.get_param('robot_description_semantic')

# make xbot model
opt = co.ConfigOptions()
opt.set_urdf(urdf)
opt.set_srdf(srdf)
opt.generate_jidmap()
opt.set_bool_parameter('is_model_floating_base', False)
opt.set_string_parameter('model_type', 'RBDL')

model = xbot.ModelInterface(opt)

# define joint range
qmin = np.full(model.getJointNum(), -2.0)
qmax = -qmin
yaml_options = '' # leave default options

In [ ]:
# make planning scene
ps = validity_check.PlanningSceneWrapper(model)
ps.startMonitor() # listen to topics / publish planning scene for visualization
ps.startGetPlanningSceneServer()  # advertise get_planning_scene srv for rviz

# add a box and two spheres to the planning scene
#ps.addBox('box', [0.1, 0.8, 0.1], Affine3(pos=[0.3, 0.2, 0.5]))
#ps.addSphere('sphere_1', 0.3, Affine3(pos=[0.0, 0.4, 0.7]))
#ps.addSphere('sphere_2', 0.2, Affine3(pos=[0.2, -0.4, 0.5]))

In [ ]:
# our validity checker checks for collisions
def model_validity_checker():
    ps.update()
    return not ps.checkCollisions()

def validity_checker(q):
    model.setJointPosition(q)
    model.update()
    return model_validity_checker()

In [ ]:
# write an ik problem to describe the manifold z = const
ee_name = 'TCP'

ik = {}

ik['solver_options'] = {}

ik['stack'] = [['ArmZ']]

ik['constraints'] = ['JointLimits', 'VelocityLimits']

ik['JointLimits'] = {
    'type': 'JointLimits'
}

ik['VelocityLimits'] = {
    'type': 'VelocityLimits',
    'limits': 0.1
}

ik['ArmZ'] = {
    'type': 'Cartesian',
    'distal_link': ee_name,
    'indices': [2]
}

ik_str = yaml.dump(ik)

# constraint from cartesian interface solver
ci = pyci.CartesianInterface.MakeInstance('OpenSot', ik_str, model, 1.0)
constr = constraints.CartesianConstraint(ci)

# create planner
planner = planning.OmplPlanner(constr, qmin, qmax, yaml_options)

In [ ]:
# define start state (joint space)
start = [0, 1, 1, 0, 0]

# set constraint to pass through the start state
model.setJointPosition(start)
model.update()
constr.reset()

# check constraint consistency
print 'Constraint value at q = start is {}'.format(constr.function(start))

# Check start state is valid
print 'Start state valid: {}'.format(validity_checker(start))
print ps.getCollidingLinks()

# Publish it to a marker array
start_viz = visual_tools.RobotViz(model, '/teleop_example/start', color=[0, 0, 1, 0.5])
start_viz.publishMarkers(ps.getCollidingLinks())

In [ ]:
# find a goal state which is constraint consistent by projection
goal = [-1, 0, 0, 1, 1]
goal = constr.project(goal)

# check constraint consistency
print 'Constraint value at q = goal is {}'.format(constr.function(goal))

print 'Goal state valid: {}'.format(validity_checker(goal))
print ps.getCollidingLinks()
goal_viz = visual_tools.RobotViz(model, '/teleop_example/goal', color=[0, 1, 0, 0.5])
goal_viz.publishMarkers(ps.getCollidingLinks())

In [ ]:
# Execute planner
planner_type = 'RRTstar'
timeout = 3.0
threshold = 0.0

planner.setStartAndGoalStates(start, goal, threshold)
planner.setStateValidityPredicate(validity_checker)
success = planner.solve(timeout, planner_type)

print 'Planner output : {}'.format(success)

solution = np.array(planner.getSolutionPath()).transpose()

In [ ]:
# Plot solution
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams.update({'figure.figsize': [14, 8],
                     'font.size': 16})

plt.plot(np.full_like(start, 0), start, '.', markersize=16)
plt.plot(np.full_like(goal, solution.shape[1]-1), goal, '.', markersize=16)
plt.plot(solution.transpose(), '-')
plt.grid()
plt.xlabel('Sample')
plt.ylabel('Joint position [rad]')
plt.legend()

In [ ]:
# Play solution a number of times..
viz = visual_tools.RobotViz(model, 
                            '/teleop_example/solution', 
                            color=[0.5, 0, 0.5, 0.5])

ntimes = 10
duration = 3.0
dt = duration/solution.shape[1]

for _ in range(ntimes):
    
    for i in range(solution.shape[1]):
        
        q = solution[:, i]
        model.setJointPosition(q)
        model.update()
        viz.publishMarkers()
        rospy.sleep(dt)